In [1]:
import torch
from torchvision.models import resnet34
from torchvision.datasets import CIFAR100
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset, ChainDataset
from torchvision.transforms import RandomHorizontalFlip, Compose
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from typing import Any


class Submean(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        return None
    """
    def __call__(self, tensor):
    def forward(self, tensor):
    둘 다 가능.
    근데, 처음에 한 번 적용되고 끝이 아니라
    데이터셋에 접근 할 때마다 계속 적용되는 것 같음.
    """
    def __call__(self, tensor):
        # Subtract the mean from each pixel along each channel
        # print("input", tensor.shape)
        _mean = tensor.mean(axis=(1, 2))
        # print("mean value before : ", _mean)
        tensor = tensor - _mean[:, None, None]
        # print("mean value after : ", tensor.mean(axis=(1, 2)))
        
        return tensor
    
    # def forward(self, tensor):
    #     return tensor
    
class SubtractMean_and_HorizontalFlip:
    def __init__(self, root, _dataset_name="CIFAR100"):
        self.transform_flip = Compose([
            ToTensor(),
            RandomHorizontalFlip(p=1.0),
        ])
        self.transform_default = Compose([
            ToTensor(),
            Submean(),
        ])
        if _dataset_name == "CIFAR100":
            self.ref_training_data = datasets.CIFAR100(
                root=root,
                train=True,
                download=True,
                transform=self.transform_default,
            )
            self.ref_test_data = datasets.CIFAR100(
                root=root,
                train=False,
                download=True,
                # transform=self.transform_default,
                transform=ToTensor(),
                
            )
            
        # Make copies
        self.training_data1 = self.ref_training_data
        self.training_data2 = self.ref_training_data
        
        # Apply Flip transform
        self.training_data2.transform = self.transform_flip

        # Merge datasets
        self.training_data = ConcatDataset([self.training_data1, self.training_data2])
        
        # Copy classes data
        self.classes = self.ref_training_data.classes
        self.training_data.classes = self.classes
        
        self.class_to_idx = self.ref_training_data.class_to_idx
        self.training_data.class_to_idx = self.class_to_idx
        
        return 

    def unpack(self):
        print("SubtractMean_and_HorizontalFlip")
        return self.training_data, self.ref_test_data


"""
subtract mean 적용 한 것.

아니 왜 training set에서는 sub mean이 잘 되는데, test set에서는 sub mean하면 안되는거야?

"""

# Usage
batch_size = 256
_dataset = SubtractMean_and_HorizontalFlip('data', 'CIFAR100')
training_data, test_data = _dataset.unpack()

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Initialize ResNet34 model
model = resnet34(pretrained=False, num_classes=100).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)


Files already downloaded and verified
Files already downloaded and verified
SubtractMean_and_HorizontalFlip


/home/lee/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lee/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    # Evaluation loop
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss /= len(test_loader)
    test_acc = correct / total

    # Print epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
    print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc*100:.2f}%")
    print("-" * 50)

# Save the trained model
torch.save(model.state_dict(), "resnet34_cifar100.pth")
print("Trained model saved successfully.")


Epoch 1/20:
Train Loss: 3.6400 | Train Acc: 14.93%
Test Loss: 3.4300 | Test Acc: 18.37%
--------------------------------------------------
Epoch 2/20:
Train Loss: 2.8043 | Train Acc: 29.19%
Test Loss: 3.2005 | Test Acc: 24.06%
--------------------------------------------------
Epoch 3/20:
Train Loss: 2.2785 | Train Acc: 39.97%
Test Loss: 2.9146 | Test Acc: 30.39%
--------------------------------------------------
Epoch 4/20:
Train Loss: 1.8403 | Train Acc: 49.77%
Test Loss: 3.1405 | Test Acc: 29.15%
--------------------------------------------------
Epoch 5/20:
Train Loss: 1.4305 | Train Acc: 59.51%
Test Loss: 3.0664 | Test Acc: 32.10%
--------------------------------------------------
Epoch 6/20:
Train Loss: 1.0630 | Train Acc: 68.83%
Test Loss: 3.5527 | Test Acc: 30.13%
--------------------------------------------------
Epoch 7/20:
Train Loss: 0.7578 | Train Acc: 76.95%
Test Loss: 3.6140 | Test Acc: 31.51%
--------------------------------------------------
Epoch 8/20:
Train Loss: 0.5

In [3]:
model.named_modules

<bound method Module.named_modules of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru